In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.svm import SVC,SVR
import os
import sys
from MFTreeSearchCV.MFTreeSearchCV import *
from mf.mf_func import *
import scipy as sp
import scipy.io as scio
import pickle as pkl
from multipolyfit.multipolyfit import multipolyfit
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model 
import matlab
import matlab.engine as meng
import pickle

In [2]:
# Start MATLAB through MATLAB Engine API
eng = meng.connect_matlab()
eng.cd('../mountain_car/')# Location of simulator

# setup:
meng.find_matlab()
eng.setup_mc(nargout=0)
eng.query_simulator(-0.52, -0.0297) # Sanity check, Answer should be around -1

-0.9726920303599369

In [3]:
# Load Data: All of it
RHO_mat = scio.loadmat('data/mountain_car/RHO.mat')
X0_mat = scio.loadmat('data/mountain_car/X0.mat')
V0_mat = scio.loadmat('data/mountain_car/V0.mat')

RHO = RHO_mat['RHO']
X0 = X0_mat['X0']
V0 = V0_mat['V0']

# Data for violations:
rho_r_mat = scio.loadmat('data/mountain_car/rho_r.mat')
xr_mat = scio.loadmat('data/mountain_car/xr.mat')
vr_mat = scio.loadmat('data/mountain_car/vr.mat')

rho_r = rho_r_mat['rho_r']
xr = xr_mat['xr']
vr = vr_mat['vr']

# Data for successes:
rho_g_mat = scio.loadmat('data/mountain_car/rho_g.mat')
xg_mat = scio.loadmat('data/mountain_car/xg.mat')
vg_mat = scio.loadmat('data/mountain_car/vg.mat')

rho_g = rho_g_mat['rho_g']
xg = xg_mat['xg']
vg = vg_mat['vg']

Nsamples = len(X0.T)
assert(Nsamples == len(V0.T))
assert(Nsamples*Nsamples == len(RHO[:,0]))
print(Nsamples)
DEG = 4 # Degree of the polynomial that is being fit to the data
SET_APPROX = 0
disc_z = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # discretizing z
step = 0.1
ndisc = len(disc_z)
print(ndisc)
fidel_dim = 2

# Regressors:
classifiers = [
    SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

100
10


In [4]:
# Constructing function approximations using polyfit:
fname_approx_polyfit = "Approximations/multipolyfit_approx.pkl"
fname_approx_logreg = "Approximations/logreg_approx.pkl"

rsv_approxs_polyfit = dict() # List containing poly-fit functions that approximate the robust satisfaction value
rsv_approxs_logreg = dict() # List containing poly-fit functions that approximate the robust satisfaction value
    
def construct_approx_polyfit():
    for x0_idx in disc_z:
        for v0_idx in disc_z: 
            idx = ndisc*(x0_idx*10 - 1) + v0_idx*10
            x0_num = int((x0_idx + step)*Nsamples)
            v0_num = int((v0_idx + step)*Nsamples)
            assert(x0_num <= Nsamples)
            if(v0_num > Nsamples):
                print(v0_num)
                assert(v0_num <= Nsamples)
            x0_indices = np.random.choice(Nsamples, x0_num, replace=False)
            v0_indices = np.random.choice(Nsamples, v0_num, replace=False)
            rho0_indices = [Nsamples*(x_ii-1)+v_ii  for x_ii in x0_indices for v_ii in v0_indices]
            x0_samples = [X0.T[ii,0] for ii in x0_indices] # Indices for x0 from the original data
            v0_samples = [V0.T[ii,0] for ii in v0_indices] # Indices for v0 from the original data
            assert(len(rho0_indices) == len(x0_indices)*len(v0_indices))
            x0 = np.array([[xi, vi] for xi in x0_samples for vi in v0_samples]) # Combined x0 and v0
            rho = np.array([RHO[ii,0] for ii in rho0_indices])
            assert(len(x0) == len(rho))
            
            model = multipolyfit(x0, rho, DEG, model_out = True)
            print(model)
            rsv_approxs_polyfit[idx] = model
    # pkl.dump(rsv_approxs, open(fname_approx_polyfit, "wb"))

zidx = lambda z1,z2: ndisc*(z1*10)+z2*10
# Constructing function approximations via tree search:
n_x0_samples = []
n_v0_samples = []
def construct_approx_logreg():
    for x0_idx in disc_z:
        for v0_idx in disc_z:
            idx = zidx(x0_idx, v0_idx)
            x0_num = int((x0_idx + step)*Nsamples)
            v0_num = int((v0_idx + step)*Nsamples)
            assert(x0_num <= Nsamples)
            if(v0_num > Nsamples):
                print(v0_num)
                assert(v0_num <= Nsamples)
            x0_indices = np.random.choice(Nsamples, x0_num, replace=False)
            v0_indices = np.random.choice(Nsamples, v0_num, replace=False)
            rho0_indices = [Nsamples*(x_ii-1)+v_ii  for x_ii in x0_indices for v_ii in v0_indices]
            x0_samples = [X0.T[ii,0] for ii in x0_indices] # Indices for x0 from the original data
            v0_samples = [V0.T[ii,0] for ii in v0_indices] # Indices for v0 from the original data
            n_x0_samples.append(len(x0_samples))
            n_v0_samples.append(len(v0_samples))
            assert(len(rho0_indices) == len(x0_indices)*len(v0_indices))
            x0 = np.array([[xi, vi] for xi in x0_samples for vi in v0_samples]) # Combined x0 and v0
            rho = np.array([RHO[ii,0] for ii in rho0_indices])
            assert(len(x0) == len(rho))
            
            X_train, X_test, y_train, y_test = train_test_split(x0, rho)
            reg = linear_model.SGDRegressor()
            reg.fit(X_train, y_train)
            print(reg.score(X_test, y_test))
            rsv_approxs_logreg[int(idx)] = reg
    # pkl.dump(rsv_approxs, open(fname_approx_logreg, "wb"))
    
            

In [ ]:
# sanity check:
xopt = [-0.52, -0.0297]
classif = classifiers.copy()
xopt_arr = np.array(xopt).reshape(1,-1)
x0_idx = 0.1
v0_idx = 0.1
rsv_approxs_logreg = dict()
idx = zidx(x0_idx, v0_idx)
print(int(idx))
x0_num = int((x0_idx + step)*Nsamples)
v0_num = int((v0_idx + step)*Nsamples)
assert(x0_num <= Nsamples)
if(v0_num > Nsamples):
    print(v0_num)
    assert(v0_num <= Nsamples)
x0_indices = np.random.choice(Nsamples, x0_num, replace=False)
v0_indices = np.random.choice(Nsamples, v0_num, replace=False)
rho0_indices = [Nsamples*(x_ii-1)+v_ii  for x_ii in x0_indices for v_ii in v0_indices]
x0_samples = [X0.T[ii,0] for ii in x0_indices] # Indices for x0 from the original data
v0_samples = [V0.T[ii,0] for ii in v0_indices] # Indices for v0 from the original data
assert(len(rho0_indices) == len(x0_indices)*len(v0_indices))
x0 = np.array([[xi, vi] for xi in x0_samples for vi in v0_samples]) # Combined x0 and v0
rho = np.array([RHO[ii,0] for ii in rho0_indices])
assert(len(x0) == len(rho))

X_train, X_test, y_train, y_test = train_test_split(x0, rho)
print(len(X_train))
reg1 = classif[-1]
reg1.fit(X_train, y_train)
print(reg1.score(X_test, y_test))
rsv_approxs_logreg[int(idx)] = reg1
print(reg1.predict(xopt_arr))


# ssecond one:
x0_idx = 0.9
v0_idx = 0.5
classif = classifiers.copy()
idx = zidx(x0_idx, v0_idx)
print(int(idx))
x0_num = int((x0_idx + step)*Nsamples)
v0_num = int((v0_idx + step)*Nsamples)
assert(x0_num <= Nsamples)
if(v0_num > Nsamples):
    print(v0_num)
    assert(v0_num <= Nsamples)
x0_indices = np.random.choice(Nsamples, x0_num, replace=False)
v0_indices = np.random.choice(Nsamples, v0_num, replace=False)
rho0_indices = [Nsamples*(x_ii-1)+v_ii  for x_ii in x0_indices for v_ii in v0_indices]
x0_samples = [X0.T[ii,0] for ii in x0_indices] # Indices for x0 from the original data
v0_samples = [V0.T[ii,0] for ii in v0_indices] # Indices for v0 from the original data
assert(len(rho0_indices) == len(x0_indices)*len(v0_indices))
x0 = np.array([[xi, vi] for xi in x0_samples for vi in v0_samples]) # Combined x0 and v0
rho = np.array([RHO[ii,0] for ii in rho0_indices])
assert(len(x0) == len(rho))

X_train, X_test, y_train, y_test = train_test_split(x0, rho)
print(len(X_train))
reg2 = classif[-1]
reg2.fit(X_train, y_train)
print(reg2.score(X_test, y_test))
rsv_approxs_logreg[int(idx)] = reg2


print(reg2.predict(xopt_arr))

In [5]:
#Calling functions to construct the models:
construct_approx_polyfit()
construct_approx_logreg()

<function mk_model.<locals>.model at 0x116b23950>
<function mk_model.<locals>.model at 0x116b23598>
<function mk_model.<locals>.model at 0x116b23a60>
<function mk_model.<locals>.model at 0x116b23620>
<function mk_model.<locals>.model at 0x116b236a8>
<function mk_model.<locals>.model at 0x116b23ae8>
<function mk_model.<locals>.model at 0x116b23510>
<function mk_model.<locals>.model at 0x116b239d8>
<function mk_model.<locals>.model at 0x116b232f0>
<function mk_model.<locals>.model at 0x116b237b8>
<function mk_model.<locals>.model at 0x11c1ce158>
<function mk_model.<locals>.model at 0x11c1ce0d0>
<function mk_model.<locals>.model at 0x11c1ce048>
<function mk_model.<locals>.model at 0x11c1ce1e0>
<function mk_model.<locals>.model at 0x11c1ce268>
<function mk_model.<locals>.model at 0x11c1ce2f0>
<function mk_model.<locals>.model at 0x11c1ce400>
<function mk_model.<locals>.model at 0x11c1ce488>
<function mk_model.<locals>.model at 0x11c1ce510>
<function mk_model.<locals>.model at 0x11c1ce598>


/Users/apurvabadithela/Documents/bbopt/TreeSearch/multipolyfit/multipolyfit/core.py:66: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  beta = linalg.lstsq(A, y)[0]


<function mk_model.<locals>.model at 0x11c1ce9d8>
<function mk_model.<locals>.model at 0x11c1ceae8>
<function mk_model.<locals>.model at 0x11c1ce950>
<function mk_model.<locals>.model at 0x11c1ce7b8>
<function mk_model.<locals>.model at 0x11c1cea60>
<function mk_model.<locals>.model at 0x11c1ceb70>
<function mk_model.<locals>.model at 0x11c1cebf8>
<function mk_model.<locals>.model at 0x11c1ced08>
<function mk_model.<locals>.model at 0x11c1ced90>
<function mk_model.<locals>.model at 0x11c1cef28>
<function mk_model.<locals>.model at 0x11c1ceea0>
<function mk_model.<locals>.model at 0x11c1cee18>
<function mk_model.<locals>.model at 0x11c1cec80>
<function mk_model.<locals>.model at 0x116b4a048>
<function mk_model.<locals>.model at 0x116b4a1e0>
<function mk_model.<locals>.model at 0x116b238c8>
<function mk_model.<locals>.model at 0x116b23840>
<function mk_model.<locals>.model at 0x116b4a0d0>
<function mk_model.<locals>.model at 0x116b4a2f0>
<function mk_model.<locals>.model at 0x116b4a400>


In [6]:
# Constructing function approximations of the robust satisfaction value at multiple fidelities:
# Use the polyfit function to create approximations:
# Using discrete fidelity bounds of 1-D fidelity: z = [0.0 0.1, 0.2, 0.3, ...., 0.9]. z \in [0,0.1) is evaluated at fidelity 0.0. 
# z = 1 is fidelity at ground truth value, and we run the simulator for that. not given with the approximations
def find_z_closest(z, fidel_dim):
    assert(len(z)==fidel_dim)
    z1 = 0 # Normalized fidelity for x0
    z2 = 0 # Normalized fidelity for  v0
    for ii in range(1, len(disc_z)):
        if(z[0]>=disc_z[ii-1] and z[0]<disc_z[ii]):
            z1 = disc_z[ii-1]
        if(z[1]>=disc_z[ii-1] and z[1]<disc_z[ii]):
            z2 = disc_z[ii-1]
    if(z[0] == 0.9):
        z1 = 0.9
    if(z[1] == 0.9):
        z2 = 0.9
    return z1, z2

def get_approx_polyfit(approx_id):
    rsv_approxs = rsv_approxs_polyfit.copy()
    return rsv_approxs[approx_id]

def get_approx_logreg(approx_id):
    rsv_approxs = rsv_approxs_logreg.copy()
    return rsv_approxs[approx_id]

def retrieve_approximation(z, fidel_dim, approx_type):
    z1, z2 = find_z_closest(z, fidel_dim)
    if (z1 < 1):
        z1_id = z1
    if (z2 < 1):
        z2_id = z2
    approx_id = int(zidx(z1,z2))
    # print("approx id: "+str(approx_id))
    if (approx_type == "logreg"):
        approximation = get_approx_logreg(approx_id)
    if (approx_type == "polyfit"):
        approximation = get_approx_polyfit(approx_id)
    return approximation
        


In [7]:
# high fidelity model
def query_sim(x):
    xarr = convert_to_arr_shape(x)
    x0 = float(xarr[0,0])
    v0 = float(xarr[0,1])
    rsv = eng.query_simulator(x0, v0)
    return -rsv
def rsv_function(z,x, approx_type):
    # """ Computes the rsv function. """
    func_computation = 0
    if(approx_type == "logreg"):
        func_computation = rsv_function_logreg(x,z)
    if (approx_type == "polyfit"):
        func_computation = rsv_function_polyfit(x,z)
    return -func_computation

def rsv_function_logreg(x, z):
    #""" Alternative form for the rsv function. """
    z_arr = convert_to_arr_shape(z)
    x_arr = convert_to_arr_shape(x)
    approximation = retrieve_approximation(z_arr[0], fidel_dim, "logreg")
    func_computation = approximation.predict(x_arr)
    return func_computation

def rsv_function_polyfit(x, z):
    #""" Alternative form for the branin function. """
    approximation = retrieve_approximation(z, fidel_dim, "polyfit")
    return approximation

def get_mf_rsv_function(fidel_dim, approx_type):
    #""" Returns the rsv function as a multifidelity function. ""
    
    def mf_rsv_obj(x, z):
    #""" Wrapper for the MF rsv objective.""" 
        assert len(z) == fidel_dim
        if (z[0]==1 or z[1]==1):
            return query_sim(x)
        else:
            if approx_type == "polyfit":
                return rsv_function(z,x,"polyfit")
            else:
                return rsv_function(z,x,"logreg")
    
    # Other data
    opt_fidel = np.ones((fidel_dim)) # For now, exclude the ground truth
    fidel_bounds = [[0, 1]] * fidel_dim
    opt_pt = np.array([-0.52, -0.0297])
    domain_bounds = [[-1, 0.6], [-0.42, 0.42]] # for x = [x0_bound, v0_bound]
    return mf_rsv_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds

# Function to get cost function:
def _get_mf_cost_function(fidel_bounds, is_0_1):
    #""" Returns the cost function. fidel_bounds are the bounds for the fidelity space
    #  and is_0_1 should be true if fidel_bounds is [0,1]^p. """
    fidel_dim = len(fidel_bounds)
    if fidel_dim == 1:
        fidel_powers = [2]
    elif fidel_dim == 2:
        fidel_powers = [3, 2]
    elif fidel_dim == 3:
        fidel_powers = [3, 2, 1.5]
    else:
        fidel_powers = [3] + list(np.linspace(2, 1.2, fidel_dim-1))
    # Define the normalised
    def _norm_cost_function(norm_z):
    # The cost function with normalised coordinates. """
        min_cost = 0.05
        return min_cost + (1-min_cost) * np.power(norm_z, fidel_powers).sum()
    
    # Now return based on whether or not is_0_1
    ret = (_norm_cost_function if is_0_1 else
           lambda z: _norm_cost_function(map_to_cube(z, fidel_bounds)))
    return ret

# Function to convert to array that can be fed into regressor from list:
def convert_to_arr_shape(opt_val):
    opt_val_arr = np.array(opt_val).reshape(1,-1)
    return opt_val_arr
def get_mf_rsv_as_mfof(fidel_dim, approx_type):
#""" Wrapper for get_mf_rsv_function which returns as a mfof. """
    mf_rsv_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds = get_mf_rsv_function(fidel_dim, approx_type)
    fidel_cost_function = _get_mf_cost_function(fidel_bounds, True) # Figure out this line

    opt_val = mf_rsv_obj(opt_fidel, opt_pt)
    return MFOptFunction(mf_rsv_obj, fidel_cost_function, fidel_bounds, domain_bounds,
                       opt_fidel, vectorised=False, opt_pt=opt_pt, opt_val=opt_val)

In [8]:
# Testing RSV function:
fidel_dim = 2
mf_rsv_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds = get_mf_rsv_function(fidel_dim, "logreg")
z1 = [0.1, 0.1]
x1 = [0.1, 0.2]
z2 = [-0.5, -0.3]
x2 = [0.1, 0.2]
xopt = [-0.52, -0.0297]
zopt = [1,1]

# print(mf_rsv_obj(xopt, zopt)) # sanity check; should be 0.972
ztest1 = [0.1, 0.1]
zid1 = zidx(ztest1[0], ztest1[1])
print(zid1)
print(mf_rsv_obj(xopt, ztest1)) # 
ztest2 = [0.1, 0.9]
print(mf_rsv_obj(xopt, ztest2)) # 
ztest3 = [0.9, 0.9]
print(mf_rsv_obj(xopt, ztest3)) # 


11.0
[0.070874]
[0.04495125]
[0.03591332]


{0: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
              eta0=0.01, fit_intercept=True, l1_ratio=0.15,
              learning_rate='invscaling', loss='squared_loss', max_iter=1000,
              n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
              warm_start=False),
 1: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
              eta0=0.01, fit_intercept=True, l1_ratio=0.15,
              learning_rate='invscaling', loss='squared_loss', max_iter=1000,
              n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
              warm_start=False),
 2: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
              eta0=0.01, fit_intercept=True, l1_ratio=0.15,
              learning_rate='invscaling', l

In [13]:
# Using MF Optimization:
mf_rsv_opt = get_mf_rsv_as_mfof(fidel_dim, "logreg")
noise_var = 0
sigma = np.sqrt(noise_var)

# Running a single experiment:
NUM_EXP = 1
EXP_NAME = "RSV_Mountain_Car"
def run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename):
    R = []
    T = []
    Xarr = []
    for t in times:
        budget = t*mfobject.opt_fidel_cost
        t1 = time.time()
        MP = MFPOO(mfobject=mfobject, nu_max=nu, rho_max=rho, total_budget=budget, sigma=sigma, 
                   C=C, mult=0.5, tol = 1e-3, Randomize = False, Auto = True, unit_cost=t0 )
        MP.run_all_MFHOO()
        X, E = MP.get_point()
        t2 = time.time()

        R = R + [E]
        T = T + [MP.cost]
        Xarr = Xarr + [X]
    print(str(MP.cost) + ' : ' + str(E))
    #print 'Total HOO Queries: ' + str(MP.t) 
    np.save(filename,R)
    return np.array(R),np.array(T), np.array(Xarr)

In [12]:
a = -1
b = 0.6
c = -0.42
d = 0.42
rho_analysis = 0.25
nu_analysis = 4*(b-a)**2 + 4*(d-c)**2


In [14]:
# Running experiments:
# times = [10,20,50,75,100,150,175, 200]
# mfobject = get_noisy_mfof_from_mfof(mf_rsv_opt, noise_var)
mfobject = mf_rsv_opt
times = [10, 20, 50]
nu = nu_analysis
rho = 0.95
C = 0.1
t0 = mfobject.opt_fidel_cost


NT = str(time.time())
print('Running Experiment 1: ')
filename = 'MFHOO' + EXP_NAME + '_' + NT + '_' + '1.npy'
R,T, Xarr = run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename)
result = R

print('Finished eXP 1')
for i in range(1,NUM_EXP):
    print('Running Experiment' + str(i+1) + ': ')
    filename = 'MFHOO' + EXP_NAME + '_' + NT + '_' + str(i+1) + '.npy'
    R,T, Xarr = run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filename)
    result = np.vstack([result,R])


# df = pd.DataFrame(dfdic)
# df.to_csv(filename) 


Running Experiment 1: 
Auto Init: 
C: 0.009300477687407029
nu: 0.009300477687407029
Budget Remaining: 18.25965691566467
Number of MFHOO Instances: 5
Budget per MFHOO Instance:1.7019313831329341
Running SOO number: 1 rho: 0.95 nu: 0.009300477687407029
Done!
Running SOO number: 2 rho: 0.9378956176563621 nu: 0.009300477687407029
Done!
Updating C
C: 0.01302066876236984
nu_max: 0.01302066876236984
Running SOO number: 3 rho: 0.9180634032924686 nu: 0.01302066876236984
Done!
Updating C
C: 0.018228936267317773
nu_max: 0.018228936267317773
Running SOO number: 4 rho: 0.8796481896190089 nu: 0.018228936267317773
Done!
Updating C
C: 0.02552051077424488
nu_max: 0.02552051077424488
Running SOO number: 5 rho: 0.7737809374999998 nu: 0.02552051077424488
Done!
Updating C
C: 0.035728715083942826
nu_max: 0.035728715083942826
Auto Init: 
C: 0.009300477687407029
nu: 0.009300477687407029
Budget Remaining: 37.759758243560796
Number of MFHOO Instances: 10
Budget per MFHOO Instance:1.8259758243560795
Running SOO 

/Users/apurvabadithela/opt/anaconda3/envs/falsification/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/Users/apurvabadithela/opt/anaconda3/envs/falsification/lib/python3.6/site-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
mu = np.mean(result,axis = 0)
std = np.std(result,axis = 0)
result = mfobject.opt_val - mu

filename = './data/mountain_car/MFHOO_' + EXP_NAME + '_' + NT + '_' + '.csv'
dfdic = {}
dfdic['Capital'] = np.array(times)
dfdic['Value'] = result
dfdic['Std'] = std

In [ ]:
pickle.dump(dfdic, open('SGDRegressor_t_50', 'wb'))

In [17]:
result
Xarr

array([list([array([ 0.378125  , -0.40523437]), array([ 0.203125  , -0.41835937]), array([0.10078125, 0.31541016]), array([0.10078125, 0.31766602]), array([0.10078125, 0.31602539])]),
       list([array([ 0.303125  , -0.41917969]), array([ 0.303125  , -0.41507812]), array([0.10078125, 0.31541016]), array([0.10234375, 0.31520508]), array([0.10390625, 0.31520508]), array([0.10234375, 0.31520508]), array([0.10078125, 0.31541016]), array([0.10078125, 0.30228516]), array([0.10078125, 0.31602539]), array([0.10078125, 0.31520508])]),
       list([array([ 0.303125  , -0.41753906]), array([ 0.403125  , -0.41917969]), array([0.10078125, 0.31520508]), array([0.10039062, 0.31561523]), array([0.10039062, 0.31561523]), array([0.10234375, 0.30208008]), array([0.10078125, 0.30208008]), array([0.10078125, 0.30228516]), array([0.10078125, 0.30208008]), array([0.10078125, 0.30208008]), array([0.10078125, 0.30208008]), array([0.10078125, 0.30208008]), array([0.10507813, 0.31602539]), array([0.10039062, 0.

In [ ]:
# from mf.mf_func import NoisyMFOptFunction, plot_2d_function

# def visualise_mfof(mfof):
#     """ Visualises the mfof object. """
#     plot_func = mfof.eval_multiple
#     _, ax, plt = plot_2d_function(plot_func,
#                                np.array([mfof.fidel_bounds[0], mfof.domain_bounds[0]]),
#                                x_label='fidel', y_label='domain')
#     ax.scatter(mfof.opt_fidel, mfof.opt_pt, mfof.opt_val, c='r', s=100)
#     plt.show()

# visualise_mfof(mf_rsv_opt) # visualize mfof function

In [ ]:
np.load("MFHOORSV_Mountain_Car_1597098498.912148_1.npy")
T
opt_val = mf_rsv_obj(opt_fidel, opt_pt)


In [ ]:
# Quit the MATLAB engine:
eng.quit()